<h3> RA response PRS prediction project </h3>

In [2]:
import numpy as np
import pandas as pd
import magenpy as mgp
# from pandas_genomics.scalars import Genotype


Read the chr22.gen file and conever to .bed file

In [3]:
chr22_geno_path = "data/DREAM_RA_Responders_GenProbData/Training_chr22.gen"
df = pd.read_csv(chr22_geno_path, sep=' ', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,8114,8115,8116,8117,8118,8119,8120,8121,8122,8123
0,22,rs915674,16053624,A,G,0.02,0.23,0.75,0.02,0.23,...,0.02,0.23,0.75,0.02,0.23,0.75,0.02,0.23,0.75,NaN
1,22,rs915677,16053758,A,G,0.00,0.13,0.87,0.00,0.13,...,0.00,0.13,0.87,0.00,0.13,0.87,0.00,0.13,0.87,NaN
2,22,rs9604721,16054713,C,T,0.93,0.07,0.00,0.93,0.07,...,0.93,0.07,0.00,0.93,0.07,0.00,0.93,0.07,0.00,NaN
3,22,rs4389403,16055070,A,G,0.01,0.19,0.79,0.01,0.19,...,0.01,0.19,0.79,0.01,0.19,0.79,0.01,0.19,0.79,NaN
4,22,rs9617528,16061016,C,T,0.07,0.38,0.55,0.07,0.38,...,0.07,0.38,0.55,0.07,0.38,0.55,0.07,0.38,0.55,NaN


In [4]:
# get the patient ID from the full clinical data.
clinical_full = pd.read_csv("data/DREAM_RA_Responders_PhenoCov_Full.txt", sep=' ', header=None)
full_ID = np.array(clinical_full[0].shift(-1))[:-1]
print("number of patients: ", len(full_ID))
full_ID
clinical_full             

number of patients:  2706


,0,1,2,3,4,5,6,7,8,9,10
0,ID,Response.deltaDAS,Response.NonResp,Response.EULAR,Batch,Cohort,Drug,baselineDAS,Age,Gender,Mtx
1,1877000,-2.88,1,Supernon,dream1,DREAM,infliximab,3.18,54.02,0,1
2,612000,-2.44,1,Supernon,REF,new,infliximab,4.65,61.08333,1,1
3,111000,-2.17,1,Supernon,REF,EIRA,etanercept,4.38,58,1,1
4,1780000,-2.11,1,Non,dream1,DREAM,infliximab,2.45,51.07,1,1
...,...,...,...,...,...,...,...,...,...,...,...
2702,2732000,2.76,0,Good,dream3,DREAM,adalimumab,5.19,NaN,1,1
2703,2736000,3.13,0,Intermediate,dream3,DREAM,adalimumab,6.59,NaN,1,1
2704,2744000,3.32,0,Intermediate,dream3,DREAM,adalimumab,6.74,NaN,1,1
2705,2748000,1.59,0,Intermediate,dream3,DREAM,adalimumab,5.01,NaN,1,1


In [5]:
# attempt to generate .sample file
col = [0, 1, 9]
sample = clinical_full.iloc[1:, col]
header = pd.MultiIndex.from_tuples([('ID', '0'), ('Response.deltaDAS', 'P'), ('Sex', 'D')])
sample.columns = header



path = "data/DREAM_RA_Responders_PhenoCov_Full.sample"
sample.to_csv(path, sep=' ', index=False)
sample

,ID,Response.deltaDAS,Sex
,0,P,D
1,1877000,-2.88,0
2,612000,-2.44,1
3,111000,-2.17,1
4,1780000,-2.11,1
5,825000,-1.97,1
...,...,...,...
2702,2732000,2.76,1
2703,2736000,3.13,1
2704,2744000,3.32,1


In [6]:
test = pd.read_csv("data/DREAM_RA_Responders_PhenoCov_Full.sample", sep=' ', header=[0, 1])
test

,ID,Response.deltaDAS,Sex
,0,P,D
0,1877000,-2.88,0.0
1,612000,-2.44,1.0
2,111000,-2.17,1.0
3,1780000,-2.11,1.0
4,825000,-1.97,1.0
...,...,...,...
2701,2732000,2.76,1.0
2702,2736000,3.13,1.0
2703,2744000,3.32,1.0


In [7]:
def gen_to_bed(gen_file, bed_file):
    # Read the .gen file into a pandas DataFrame
    df = pd.read_csv(gen_file, sep=' ', header=None)

    # Extract the relevant columns (e.g., chromosome, start position, end position)
    # and create a new DataFrame with the desired column names
    bed_df = pd.DataFrame()
    bed_df['chromosome'] = df[0]
    bed_df['SNP'] = df[1]
    
    for i in range(0,2706):   # patient ids from 0-2705
        pid = full_ID[i]
        bed_df[pid] =df.iloc[:, i*3+5].astype(str) + ', ' + df.iloc[:, i*3+6].astype(str) + ', ' + df.iloc[:, i*3+7].astype(str)

    # loop to fill in the genotype probabilities for the 2706 individuals 
    bed_df.to_csv(bed_file, sep=' ', index=False, header=False)


In [8]:
chr22_bed_try = "LD_chr22_training/chr22_try.bed"
gen_to_bed(chr22_geno_path, chr22_bed_try)

KeyboardInterrupt: 

In [20]:
input = pd.read_csv(chr22_bed_try, sep=' ', header=None)
input 

,0,1,2,3,4,5,6,7,8,9,...,2698,2699,2700,2701,2702,2703,2704,2705,2706,2707
0,22,rs915674,"0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75",...,"0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75","0.02, 0.23, 0.75"
1,22,rs915677,"0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87",...,"0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87","0.0, 0.13, 0.87"
2,22,rs9604721,"0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0",...,"0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0","0.93, 0.07, 0.0"
3,22,rs4389403,"0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79",...,"0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79","0.01, 0.19, 0.79"
4,22,rs9617528,"0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55",...,"0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55","0.07, 0.38, 0.55"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33740,22,rs2285395,"0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.13, 0.87","0.0, 0.12, 0.87","0.0, 0.17, 0.82",...,"0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0","0.0, 0.0, 1.0"
33741,22,rs8136603,"0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.85, 0.15, 0.0","0.85, 0.14, 0.0","0.81, 0.18, 0.01",...,"0.97, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.98, 0.02, 0.0","0.99, 0.01, 0.0"
33742,22,rs3888396,"0.0, 0.19, 0.8","0.0, 0.27, 0.72","0.0, 0.2, 0.8","0.01, 0.44, 0.55","0.0, 0.08, 0.92","0.0, 0.16, 0.83","0.0, 0.17, 0.82","0.0, 0.16, 0.83",...,"0.0, 0.15, 0.85","0.0, 0.07, 0.92","0.0, 0.06, 0.94","0.0, 0.07, 0.93","0.0, 0.08, 0.92","0.01, 0.23, 0.76","0.0, 0.06, 0.94","0.0, 0.06, 0.94","0.0, 0.1, 0.89","0.0, 0.07, 0.93"
33743,22,rs2238837,"0.75, 0.23, 0.02","0.12, 0.68, 0.2","0.57, 0.36, 0.07","0.59, 0.37, 0.04","0.43, 0.5, 0.07","0.38, 0.47, 0.15","0.38, 0.47, 0.15","0.38, 0.47, 0.15",...,"0.57, 0.4, 0.03","0.15, 0.73, 0.12","0.14, 0.75, 0.11","0.08, 0.51, 0.4","0.24, 0.67, 0.09","0.28, 0.53, 0.19","0.14, 0.75, 0.11","0.14, 0.76, 0.11","0.15, 0.75, 0.11","0.14, 0.75, 0.12"


In [6]:
# example for utilizing mgp
g_sim = mgp.GWASimulator(mgp.tgp_eur_data_path(),
                         h2=0.1)
g_sim.simulate()
g_sim.to_phenotype_table()
#          FID      IID  phenotype
# 0    HG00096  HG00096   0.795651
# 1    HG00097  HG00097   0.550914
# 2    HG00099  HG00099  -0.928486
# 3    HG00100  HG00100   0.893626
# 4    HG00101  HG00101  -0.670106
# ..       ...      ...        ...
# 373  NA20815  NA20815   0.246071
# 374  NA20818  NA20818   1.821426
# 375  NA20819  NA20819  -0.457994
# 376  NA20826  NA20826   0.954208
# 377  NA20828  NA20828   0.088412
#
# [378 rows x 3 columns]


> Reading BED file...
> Generating polygenic scores...


,FID,IID,phenotype
0,HG00096,HG00096,0.425687
1,HG00097,HG00097,-0.918492
2,HG00099,HG00099,-0.226575
3,HG00100,HG00100,-1.241405
4,HG00101,HG00101,-0.789751
...,...,...,...
373,NA20815,NA20815,1.618050
374,NA20818,NA20818,-0.314308
375,NA20819,NA20819,-0.640369
376,NA20826,NA20826,1.358444


In [7]:
print(mgp.tgp_eur_data_path())

/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/magenpy/data/1000G_eur_chr22


In [8]:
chr22_bim_try = "/home/mcb/users/yzhu439/RA_Project/LD_chr22_training/chr22_try.bim"
chr22_fam_try = "/home/mcb/users/yzhu439/RA_Project/LD_chr22_training/chr22_try.fam"

bed_df = pd.read_csv(chr22_bed_try, sep=' ', header=None)
bed_df.to_csv(chr22_bim_try, sep=' ', index=False, header=False)
bed_df.to_csv(chr22_fam_try, sep=' ', index=False, header=False)



In [21]:
out = "/home/mcb/users/yzhu439/RA_Project/LD_chr22_training/chr22_tryLD_out/"
chr22_try_path = "/home/mcb/users/yzhu439/RA_Project/LD_chr22_training"
gdl = mgp.GWADataLoader(chr22_try_path,backend='plink')
 
gdl.compute_ld(estimator='windowed',
               output_dir=out,
               window_size=3.)
# gdl.compute_ld(estimator='windowed', output_dir=out, window_size=100)

/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/magenpy/GWADataLoader.py:336: UserWarning: No BED files were found at: /home/mcb/users/yzhu439/RA_Project/LD_chr22_training
  warnings.warn(f"No BED files were found at: {bed_paths}")


TypeError: object of type 'NoneType' has no len()

<h3> Convert clinical data .txt to .sample </h3>

In [22]:
import os
def convert_txt_to_sample(file_path):
    # Read the content of the .txt file
    with open(file_path, 'r') as txt_file:
        content = txt_file.read()

    # Change the file extension to .sample
    out_path = "/home/mcb/users/yzhu439/RA_Project/sample_clinicalData/DREAM_RA_Responders_PhenoCov_Full"
    new_file_path = os.path.splitext(out_path)[0] + '.sample'

    # Write the content to the .sample file
    with open(new_file_path, 'w') as sample_file:
        sample_file.write(content)

    print(f"Conversion complete. File saved as {new_file_path}")
    
clinical_full_path = "data/DREAM_RA_Responders_PhenoCov_Full.txt"
convert_txt_to_sample(clinical_full_path)

Conversion complete. File saved as /home/mcb/users/yzhu439/RA_Project/sample_clinicalData/DREAM_RA_Responders_PhenoCov_Full.sample


<h3>Read the plink.assoc.linear</h3>


In [18]:
linear_data = pd.read_csv('plink.assoc.linear', sep='\s+')
linear_data #for merge10.bed, has a .nosex file 

,CHR,SNP,BP,A1,TEST,NMISS,BETA,STAT,P
0,0,200003,0,G,ADD,703,-0.05554,-0.7707,0.4411
1,0,200006,0,A,ADD,703,-0.02408,-0.3352,0.7376
2,0,200050,0,C,ADD,417,-0.07902,-0.1913,0.8484
3,0,200052,0,A,ADD,417,-0.07902,-0.1913,0.8484
4,0,200053,0,G,ADD,417,-0.07902,-0.1913,0.8484
...,...,...,...,...,...,...,...,...,...
1342721,26,MitoA16164,16164,G,ADD,574,0.40980,0.9125,0.3619
1342722,26,MitoC16184,16184,C,ADD,578,0.20460,0.7126,0.4764
1342723,26,MitoC16280,16280,A,ADD,169,-0.31720,-0.3048,0.7609
1342724,26,MitoG16392,16392,A,ADD,171,0.54980,0.6496,0.5168


In [47]:
# Using plink:
import magenpy as mgp
path = "data/Merged_Genotype/merged1"
gdl = mgp.GWADataLoader(path)
gdl.compute_ld(estimator='windowed',
               output_dir='data/Merged_Genotype/',
               window_size=100
               )

> Reading BED file...


Computing LD matrices:   0%|          | 0/24 [00:00<?, ?it/s]

Computing LD matrices:   4%|▍         | 1/24 [00:11<04:18, 11.22s/it]/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/dask/array/routines.py:318: PerformanceWarning: Increasing number of chunks by factor of 23
  axes=(left_axes, right_axes),


Computing LD matrices:   8%|▊         | 2/24 [18:52<4:03:35, 664.34s/it]/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/dask/array/routines.py:318: PerformanceWarning: Increasing number of chunks by factor of 25
  axes=(left_axes, right_axes),


Computing LD matrices:  12%|█▎        | 3/24 [41:54<5:47:15, 992.15s/it]/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/dask/array/routines.py:318: PerformanceWarning: Increasing number of chunks by factor of 21
  axes=(left_axes, right_axes),
/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


Computing LD matrices:  17%|█▋        | 4/24 [58:34<5:31:43, 995.20s/it]/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/dask/array/routines.py:318: PerformanceWarning: Increasing number of chunks by factor of 19
  axes=(left_axes, right_axes),
/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


In [ ]:
# chapGPT naive way of computing LD
chr22_geno_path = "data/DREAM_RA_Responders_GenProbData/Training_chr22.gen"
df_gen = pd.read_csv(chr22_geno_path, sep=' ', header=None)
df_transformed = pd.DataFrame()



In [8]:
import magenpy as mgp
import pandas as pd
ldm_0 = mgp.LDMatrix.from_path("data/Merged_Genotype/ld/chr_1")
print(pd.DataFrame(ldm_0[0]))
pd.DataFrame(ldm_0.z_array)

           0
0   0.994695
1  -0.242979
2   0.367144
3   0.089812
4   0.083862
..       ...
95 -0.076273
96 -0.071216
97  0.044402
98 -0.036434
99  0.040291

[100 rows x 1 columns]


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.994695,-0.242979,0.367144,0.089812,0.083862,0.212863,0.041536,0.045998,-0.044178,0.073736,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.242979,0.989389,-0.161772,0.470516,-0.131669,-0.138580,-0.040055,0.055405,0.318803,0.002061,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.367144,-0.161772,0.999998,-0.290074,0.655035,0.840125,0.579983,0.007434,0.013942,0.084794,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.089812,0.470516,-0.290074,0.997347,-0.217304,-0.237682,-0.296156,0.023672,0.212040,-0.127402,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.083862,-0.131669,0.655035,-0.217304,0.994691,0.807780,0.764433,0.072185,-0.025793,0.188245,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22889,0.016411,0.004080,0.084702,0.085127,0.016340,-0.039284,0.100573,-0.017008,-0.055851,0.003261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22890,0.026202,0.026955,0.074140,0.104467,0.012133,0.090469,-0.001504,-0.067641,-0.005455,0.088095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22891,0.024879,-0.013678,0.136113,0.057195,0.017412,-0.013057,-0.026552,-0.026343,0.009709,-0.027324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22892,-0.051988,0.056451,-0.005732,-0.007021,-0.044434,-0.030586,-0.015376,0.058487,0.005064,0.035679,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h3> After June 9th, LD computation and validation </h3>

<h5> Read the created .ld files </h5>

In [3]:
import pandas as pd
import numpy as np
dream1_ld_path = "qced_genotype_out/dream1.ld"
dream2_ld_path = "qced_genotype_out/dream2.ld"
dream3_ld_path = "qced_genotype_out/dream3.ld"

d1_ld = pd.read_csv(dream1_ld_path, sep=r'\s+|\t+')
d2_ld = pd.read_csv(dream2_ld_path, sep=r'\s+|\t+')
d3_ld = pd.read_csv(dream3_ld_path, sep=r'\s+|\t+')

/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [4]:
'''
todo - June 9th
trim data by removing chr23 onward (sex and mitochodrial chromosomes)
check if the ld computed is consistented between 3 different dream data
'''
# got the human chromosome 1-22 
d1_ld_trim = d1_ld[d1_ld['CHR_A'] <= 22]
d2_ld_trim = d2_ld[d2_ld['CHR_A'] <= 22]
d3_ld_trim = d3_ld[d3_ld['CHR_A'] <= 22]

print(d1_ld_trim.shape)
print(d2_ld_trim.shape)
print(d3_ld_trim.shape)
d1_ld_trim


(1377994, 7)
(1349908, 7)
(1386914, 7)


,CHR_A,BP_A,SNP_A,CHR_B,BP_B,SNP_B,R2
0,1,995669,rs3934834,1,1008567,rs9442372,0.225404
1,1,995669,rs3934834,1,1020428,rs6687776,0.231875
2,1,1008567,rs9442372,1,1011278,rs3737728,0.496356
3,1,1008567,rs9442372,1,1021403,rs9651273,0.404016
4,1,1008567,rs9442372,1,1051029,rs11807848,0.255884
...,...,...,...,...,...,...,...
1377989,22,49470371,rs9616816,22,49512530,rs8137951,0.222576
1377990,22,49493005,rs5770992,22,49497339,rs5770820,0.352207
1377991,22,49493881,rs2040487,22,49498590,rs6010061,0.304668
1377992,22,49497339,rs5770820,22,49498590,rs6010061,0.492330


<h5> merge SNP pairs with 3 R_squared values </h5>
Merge SNP pairs in dream1,2,3 LD to a dictionary in order to <code> compare the r_squared </code> for the <code> sanme SNP pair</code>.
Then Use the <code> the largest range or distance </code> between the LD values for the same SNP pair calcualted from dream1,2,3 to compare the similarity.     
 
The <code>threshold</code> for the difference of LDs for the same SNP pair is set as <code>0.11</code>

In [5]:
LD_dict = {}
for index, row in d1_ld_trim.iterrows():
    key = (row["SNP_A"], row["SNP_B"])
    if key in LD_dict.keys():
        (LD_dict[key]).append(row["R2"])
    else:
        LD_dict[key] = [row["R2"]]

print("after merge d1_ld to dictionary ", len(LD_dict.keys()))

# merge d2_ld
for index, row in d2_ld_trim.iterrows():
    key = (row["SNP_A"], row["SNP_B"])
    if key in LD_dict.keys():
        (LD_dict[key]).append(row["R2"])
    else:
        LD_dict[key] = [row["R2"]]
print("after merge d1_ld to dictionary ",len(LD_dict.keys()))

# merge d3_ld
for index, row in d3_ld_trim.iterrows():
    key = (row["SNP_A"], row["SNP_B"])
    if key in LD_dict.keys():
        (LD_dict[key]).append(row["R2"])
    else:
        LD_dict[key] = [row["R2"]]

print("after merge d1_ld to dictionary ",len(LD_dict.keys()))

after merge d1_ld to dictionary  1377994
after merge d1_ld to dictionary  1625575
after merge d1_ld to dictionary  1690878


In [6]:
''' Use the the largest range or distance between the LD values for the same SNP pair 
calcualted from dream1,2,3 to compare the similarity. 
'''
threshold_simi = 0.11
LD_SNPs_pair = np.array(list(LD_dict.keys()))
LDnp = np.array(list(LD_dict.values()))

Simi_LDnp = np.fromiter(((max(e)-min(e)) for e in LDnp), dtype=np.float)
underThr_indices = np.where(Simi_LDnp <= 0.11)[0]

print("the threshold of difference between 3 dreams' LD is: ", threshold_simi)
print("number of SNPs pair in dream1,2,3: ", len(Simi_LDnp))
print("number of SNPs pair under the threshold: ", len(underThr_indices))


/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/home/mcb/users/yzhu439/RA_Project/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


the threshold of difference between 3 dreams' LD is:  0.11
number of SNPs pair in dream1,2,3:  1690878
number of SNPs pair under the threshold:  1429419


From the result above, for all 3 dream.bed files, arond 84.57% SNP pairs' LD are similar with that computed from other dream files. Thus we decided we can use the LD computed from dream1 to calculate the PRS.  
Uese the commend below to get the Gwas of dream1:  
<code> plink --bfile data/RA_challenge_qced_genotypes/dream1 --pheno data/utils/DREAM_pheno_FID.txt --linear hide-covar --pheno-name Response.deltaDAS </code>

In [8]:
# here the Gwas for the dream1.bed file
linear_data = pd.read_csv('plink.assoc.linear', sep='\s+')
linear_data

,CHR,SNP,BP,A1,TEST,NMISS,BETA,STAT,P
0,1,rs3094315,742429,G,ADD,176,0.08742,0.40030,0.6894
1,1,rs12562034,758311,A,ADD,176,-0.40550,-1.61400,0.1083
2,1,rs3934834,995669,A,ADD,177,-0.15230,-0.67120,0.5030
3,1,rs9442372,1008567,A,ADD,177,0.12620,0.81020,0.4189
4,1,rs3737728,1011278,A,ADD,176,0.27580,1.62500,0.1061
...,...,...,...,...,...,...,...,...,...
527055,26,MitoA16163,16163,G,ADD,175,0.83490,1.28300,0.2012
527056,26,MitoA16164,16164,G,ADD,171,-0.17600,-0.17120,0.8643
527057,26,MitoC16184,16184,C,ADD,172,0.01727,0.03149,0.9749
527058,26,MitoC16280,16280,A,ADD,169,-0.31720,-0.30480,0.7609
